# Modules

In [1]:
import sys, os, datetime
sys.path.append("../src")

import numpy as np
import pandas as pd
import optuna.integration.lightgbm as lgb

from dataset import ds_generator
from config import data_configuration
from exp_helpers import result_path, format_results

In [2]:
from dataio import get_results

## config

In [5]:
# get calucurete index
df = pd.read_pickle('data_details.pkl')
idxs = df[~(df['lack of data']) & (df['network exist']) & ~(df['other dams']) & (df['gauge available'])].index

In [11]:
model = lgb.LGBMRegressor(seed=0)
params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

/home/yoshimi/anaconda3/lib/python3.8/site-packages/optuna/integration/_lightgbm_tuner/sklearn.py:45: UserWarning: LightGBMTuner doesn't support sklearn API. Use `train()` or `LightGBMTuner` for hyperparameter tuning.
  warnings.warn(


## Prediction

In [ ]:
config_args = data_configuration(rain_obs='msm', pca_exec=False, model='lgb')

# make directory
dirname = result_path(config_args)
os.makedirs(dirname, exist_ok=True)

for (idx, te_ds, train_x, train_y, train_date, val_x, val_y, val_date,
     test_x, test_y, test_date) in ds_generator(idxs, **config_args):
    
    trains = lgb.Dataset(train_x, train_y)
    valids = lgb.Dataset(val_x, val_y)
    tests = lgb.Dataset(test_x, test_y)


    model = lgb.train(params, trains, valid_sets=valids,
                        verbose_eval=False,
                        num_boost_round=100,
                        early_stopping_rounds=5)

    predict = model.predict(test_x, num_iteration=model.best_iteration)
    
    result = format_results(predict, test_y, te_ds, test_date)
    result.to_pickle(f"{dirname}/{idx}.pkl")

  0%|          | 0/86 [00:00<?, ?it/s]

[I 2022-10-12 09:19:23,957] A new study created in memory with name: no-name-c2eaa40e-b068-4e7e-8c3c-4be8a73188ce

feature_fraction, val_score: 27.798101:  14%|#4        | 1/7 [00:02<00:15,  2.58s/it][I 2022-10-12 09:19:26,551] Trial 0 finished with value: 27.798100839739313 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 27.798100839739313.

feature_fraction, val_score: 27.798101:  29%|##8       | 2/7 [00:03<00:09,  1.87s/it][I 2022-10-12 09:19:27,928] Trial 1 finished with value: 28.69262771458083 and parameters: {'feature_fraction': 0.8999999999999999}. Best is trial 0 with value: 27.798100839739313.

feature_fraction, val_score: 27.798101:  43%|####2     | 3/7 [00:05<00:06,  1.64s/it][I 2022-10-12 09:19:29,290] Trial 2 finished with value: 29.789089978022247 and parameters: {'feature_fraction': 0.6}. Best is trial 0 with value: 27.798100839739313.

feature_fraction, val_score: 27.798101:  57%|#####7    | 4/7 [00:06<00:04,  1.48s/it][I 2022-10-12 09:19:30,536]